In [1]:
import sys
from pathlib import Path
import os 
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[4]
sys.path.append(str(root))

os.chdir(root)

In [2]:
import torch
import os
import random
import json
from pathlib import Path

import pandas as pd

from fairness_cv_project.methods.label_free_cbm.src.plots import plots
from fairness_cv_project.methods.label_free_cbm.src.utils import data_utils, utils
from fairness_cv_project.methods.label_free_cbm.src.models import cbm

In [3]:
# change this to the correct model dir, everything else should be taken care of
load_dir = "saved_models/imSitu/30_verbs/CBM/sparse/imbalanced_2/imSitu_30_imbalanced_2_imSitu_30_filtered"
path_model_dir = root / load_dir
device = "cuda" if torch.cuda.is_available() else 'cpu'


with open(root / os.path.join(load_dir, "args.txt"), "r") as f:
    args = json.load(f)
    
dataset = args["dataset"]
_, target_preprocess = data_utils.get_target_model(args["backbone"], device, root)
model = cbm.load_cbm(path_model_dir, device)

In [4]:
val_d_probe = dataset+"_val"
cls_file = data_utils.LABEL_FILES[dataset]

val_data_t = data_utils.get_data(val_d_probe, preprocess=target_preprocess)
val_pil_data = data_utils.get_data(val_d_probe)

In [5]:
with open(cls_file, "r") as f:
    classes = f.read().split("\n")

with open(os.path.join(load_dir, "concepts.txt"), "r") as f:
    concepts = f.read().split("\n")

In [6]:
def print_weights(classes, list_verbs):
    for verb in list_verbs:
        i = classes.index(verb)
        print("Output class:{} - {}".format(i, verb))
        print("Incoming weights:")
        for j in range(len(concepts)):
                print("{} [{:.4f}] {}".format(concepts[j], model.final.weight[i,j], classes[i]))
                
def save_weights_to_excel(classes, model, concepts, file_name='weights_sparse_imbalanced_2_gender.xlsx'):
    with pd.ExcelWriter(file_name) as writer:  
        all_weights = []
        for verb in classes:
            i = classes.index(verb)
            data = []
            for j in range(len(concepts)):
                if torch.abs(model.final.weight[i,j])>0.01:
                    data.append({"Concept": concepts[j], "Weight": model.final.weight[i,j].item(), "Class": classes[i]})
            df = pd.DataFrame(data)
            df.sort_values(['Class', 'Weight'], ascending=[True, False], inplace=True)
            df.to_excel(writer, sheet_name=verb, index=False)
            all_weights.append(df)
        
        # For the first sheet, concatenate all dataframes and write to the sheet
        pd.concat(all_weights).to_excel(writer, sheet_name='All Weights', index=False)